## Julia Macros : Learning through use cases

If you are reading this, you either didn't read the Julia Meta Programming docs ; or you are a masochist. The docs are correct. Nobody in their right mind should be trying to create macros. They are too difficult. I didn't believe the docs. I use macros often. What would life be like without @btime; @match; @test; @kwdef. Their authors must be of sound mind. Surely macros can't be all that bad.

I took the plunge. I wrote one. It was easy. The docs lied. I took the plunge again. I almost drowned. The docs are right. Macros are almost impossible. I must be a masochist. I thought if I studdied a few macros I would figure out the patterns and life would be good. I followed the patterns. I stuck random $'s signs and "esc" in various places in my macros. Sometimes it even worked. How many missteps would it take to figure out the patterns?

### Disclaimer: Reading this is not going to make macros easy

Reading this will help you understand why macros are hard. It may encourge you to try harder to find a function that will do a simalar job, or that copy and paste isn't as bad as you thought it was. 

### Background: Why I choose the masochistic root.

I am building a DSL for composition. This is a passion project. It's first task the rather challenging task of composing mulutimedia artworks comprising of music and images. I am hoping for a natural and expressive way for the composer to provide input to the composition process. A natural process will make for easy learning and easy iterative revisions. Nothing kills the creative flow more than hunting down coding errors and fighting the syntax to work around them. I know that if I use macros, I have less constraints on the syntax. Given the dire warnings in the docs, I wanted to know how viable it would front my API with macros, hiding most of my functions and resticting the amount of Julia syntax that the composer needs to understand. 

### Metagoal: Life lessons

As technologists, we pride ourselves in what we know. We also pride ourselves in our ability to get stuff done. When our deep knowledge propells us to get stuff done better and faster, these two sources of pride are aligned. However when we favor what we know for expediant results, we actively avoid what we don't know. This is a big deal! It hampers our personal growth. It causes conflict in teams. I could go on..., but instead I will leave you with my abstract metagoal: to find techniques to embrace unknowns; get stuff done ; and make meaningful progress. **Meaningful progress seldom comes from what you know**.

### A gentle start

Not all macros are hard. When I got lucky with my first one, it wasn't luck. If all you need is a macro that substitutes a variable or two into templated code, you should be ok.

In [1]:
"""
Enable multiply for a struct with a .data vector
"""
macro vectormult(T)
    quote
        @eval begin
            (*)(s::$T, x::Vector{Float64}) = s.data .* x
            (*)(x::Vector{Float64}, s::$T) = s.data .* x

            (*)(s::$T, x::Float64) = s.data .* x
            (*)(x::Float64, s::$T) = s.data .* x

            (*)(s::$T, x::Int) = s.data .* Float64(x)
            (*)(x::Int, s::$T) = s.data .* Float64(x)

            (*)(s::$T, x::$T) = s.data .* x.data
        end
    end
end

@vectormult

I have an aversion to copy and paste and I have lots of structs that contain a single vector and a bunch of properties. These structs are just dying to be multiplied by things. The macro above was a easy to write. It is not going to win any prizes for generic coding, but give me a break. This was my first macro and it works perfectly for what I needed it for. Here is an example.

In [2]:
import Base: *

struct Amplitude
    data::Vector{Float64}
    alpha::Float64
    beta::Float64
end

@vectormult Amplitude

a = Amplitude([3.2, 1.3], 0.5, 0.33)
2a

2-element Vector{Float64}:
 6.4
 2.6

Not exactly ground breaking stuff, but it shows that macros can be easy. What makes this one easy is:

1) It is basic template substitution. Julia substitutes T with Amplitude at compile time and produces new code that points to the Amplitude struct.
2) The act of substituation affect what this macro brings back into the main namespace. It always brings back (*).
3) T is a scalar.
4) The input to the macro (T) is not transformed by the macro. It is used as is: $T.

For any macro that conforms to the above criteria, you have nothing to fear. Go forth and mutiply, divide or do anything to avoid the dreaded copy and paste.

### Julia Expressions

There is a lot of material covering Julia expressions and the abstract syntax tree. I am not going to cover it again. When attempt a macro that violoates any of the four easy criteria, you need to be aware of what is happening to whatever you pass into your macro.

#### Get used to taking a peak at expressions

One of my early fears with macros was *print()*. There are examples that show compile time *print()* statements vs runtime *print()* statements. The compile time statements are really useful.

**Pro tip:** Anything you put before the start of the quote block is regular Julia code (free from any macro weirdness) and it only runs at compile time. 

In [3]:
macro vectormult(T)
    @show T
    quote
        @eval begin
            (*)(s::$T, x::Vector{Float64}) = s.data .* x
            (*)(x::Vector{Float64}, s::$T) = s.data .* x

            (*)(s::$T, x::Float64) = s.data .* x
            (*)(x::Float64, s::$T) = s.data .* x

            (*)(s::$T, x::Int) = s.data .* Float64(x)
            (*)(x::Int, s::$T) = s.data .* Float64(x)

            (*)(s::$T, x::$T) = s.data .* x.data
        end
    end
end

@vectormult Amplitude


T = :Amplitude


* (generic function with 325 methods)

When all you pass to a macros is a scalar value or identifier, it easy to understand how the macro sees it.

In [4]:

macro showinput(y)
	show(y)
end

@showinput Amplitude

:Amplitude

The macro sees Amplitude as the symbol :Amplitude

In [5]:
@showinput Amplitude, println

:((Amplitude, println))

The macro sees the input "Amplitude, println" as an expression representation of a tuple. This is how you can create an expression yourself.

In [6]:
x = :((Amplitude, println))

:((Amplitude, println))

The two inputs to the macro were conveniently converted to a tuple and this tuple was converted into an expression. It is no longer a tuple, so all of its tupplness was left behind when it was turned into an expression. It is no longer indexable, so it is not iterable either. You have the power of the julia programming language to operate on it, but it is pretty much powerless. Don't say nobody warned you. Macros are hard!

In [7]:
x[1]

MethodError: MethodError: no method matching getindex(::Expr, ::Int64)

Note: It is useful to know how to roundtrip things and convert an expression back to Julia objects. You use eval(). Horray! We have an indexable tuple again.

In [8]:
eval(x)[1]

Amplitude

#### Making sense of input expressions

Unless you pass one or more scalars to a macro that you use verbatim without transformation, you will need learn how to decompose expression into pieces and work with those pieces. This next section gives you some practical ways to get to grips with expressions.

In [9]:
dump(x)

Expr
  head: Symbol tuple
  args: Array{Any}((2,))
    1: Symbol Amplitude
    2: Symbol println


By dumping the expression we get a better view of what is inside an expression. All expressions look the same. They contains a head that explains what type of expression it is and args that explain what the expression operates on. The data structure is a tree. Let's make an expanded one and examine the tree.

In [10]:
exprtree = :(Amplitude, x + 1)
dump(exprtree)

Expr
  head: Symbol tuple
  args: Array{Any}((2,))
    1: Symbol Amplitude
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol x
        3: Int64 1


When you work with the contents of expressions in your macro code, do don't have the convenience of "dump". You can traverse the tree starting with the args of the root.

In [11]:
exprtree.args

2-element Vector{Any}:
 :Amplitude
 :(x + 1)

Since args is a vector we can index into it.

In [12]:
exprtree.args[2].head

:call

You may have expected to see a *+* as the head, but *+* is not an expression type. *+* is a function. All function calls live inside an expression type of *:call*. The args are the function name *+* and the literals that are being added.

In [13]:
exprtree.args[2].args

3-element Vector{Any}:
  :+
  :x
 1

If *x+1* were indeed an example of an input that your needed for a macro and you need to isolate each piece as variable like x, a operation like + and a constant like 1, you could locate these elements by position in the expression.

In [14]:
variable = exprtree.args[2].args[2]
operation = exprtree.args[2].args[1]
constant = exprtree.args[2].args[3]
(variable, operation, constant)

(:x, :+, 1)

This clearly works, and you may call me picky, but I not too keen on using, debugging or maintaining code that looks like this. I wondered whether there would be a way to parse expressions in a way that makes it easier to adapt to variations in the structure of the expression, gives good feedback to the developer when they mess up and supply the wrong input.

### A more methodical way

The example below demonstrates a more repeatable way of hunting down and structuring fragments in expressions.

In [15]:
include("src/MacroExpression.jl")



Main.MacroExpression

In [16]:
MacroExpression.parseinput([MacroExpression.BinaryFunction{Symbol, Int}], :(x+1))

1-element Vector{Any}:
 Main.MacroExpression.BinaryFunction{Symbol, Int64}(:+, (Symbol, Int64))